# Imports:

In [1]:
from CSIKit.reader import get_reader
from CSIKit.util import csitools
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join

# All File names from directory : 

In [4]:
#internet dataset
#mypath = r"data/"
#command dataset
mypath = r"pcap_files/"

file_names = [f.split(".")[0] for f in listdir(mypath) if isfile(join(mypath, f))]


# Replace inf rows, inf numbers and convert pcap to csv amplitudes files :

In [3]:
for name in file_names:
    my_reader = get_reader(mypath + f"{name}.pcap")
    csi_data = my_reader.read_file(mypath + f"{name}.pcap", scaled=True)
    csi_matrix, no_frames, no_subcarriers = csitools.get_CSI(csi_data, metric="amplitude")
    arr = csi_matrix[:,:,0,0]
    
    indx = []
    for count,i in enumerate(arr):
        if len(set(i.flat)) == 1:
            indx.append(count)
        else:
            arr[count] = np.where(i == -np.inf, np.ma.masked_invalid(i).mean(),i)
            
    arr = np.delete(arr,indx,axis=0)
    df = pd.DataFrame(arr)
    #Commnds dataset
    #df.to_csv(path_or_buf = fr"csv_files/csv_amplitude/{name}.csv" , index = False)
    #Internet dataset
    #df.to_csv(path_or_buf = fr"csv_files/csv_data_amplitude/{name}.csv" , index = False)

brushteeth1
brushteeth10
brushteeth11
brushteeth12
brushteeth13
brushteeth14
brushteeth15
brushteeth16
brushteeth17
brushteeth18
brushteeth19
brushteeth2
brushteeth20
brushteeth21
brushteeth22
brushteeth23
brushteeth24
brushteeth25
brushteeth26
brushteeth27
brushteeth28
brushteeth29
brushteeth3
brushteeth30
brushteeth31
brushteeth32
brushteeth33
brushteeth34
brushteeth35
brushteeth36
brushteeth37
brushteeth38
brushteeth39
brushteeth4
brushteeth40
brushteeth41
brushteeth42
brushteeth43
brushteeth44
brushteeth45
brushteeth46
brushteeth47
brushteeth48
brushteeth49
brushteeth5
brushteeth50
brushteeth6
brushteeth7
brushteeth8
brushteeth9
cook1
cook10
cook11
cook12
cook13
cook14
cook15
cook16
cook17
cook18
cook19
cook2
cook20
cook21
cook22
cook23
cook24
cook25
cook26
cook27
cook28
cook29
cook3
cook30
cook31
cook32
cook33
cook34
cook35
cook36
cook37
cook38
cook39
cook4
cook40
cook41
cook42
cook43
cook44
cook45
cook46
cook47
cook48
cook49
cook5
cook50
cook6
cook7
cook8
cook9
drink1
drink10
dri